## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/FakeNewsDataset/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Drop Nan Values

In [ ]:
df=df.dropna()

Get the Independent Features

In [ ]:
X=df.drop('label',axis=1)

 Get the Dependent features

In [ ]:
y=df['label']

In [ ]:
X.shape

(3474, 4)

In [ ]:
y.shape

(3474,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

Vocabulary size

In [ ]:
voc_size=5000

### Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

 Dataset Preprocessing

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[3]

'civilian kill singl us airstrik identifi'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4664, 1754, 270, 3232, 2447, 714, 441, 1013, 2515, 4403],
 [3231, 3514, 2495, 3555, 2514, 908, 3388],
 [4076, 4415, 1959, 1680],
 [4281, 691, 875, 2379, 1099, 2095],
 [4864, 2514, 2486, 313, 893, 3896, 2514, 222, 1405, 2180],
 [2198,
  4978,
  2902,
  2353,
  3322,
  1538,
  1401,
  4730,
  1035,
  1673,
  166,
  2174,
  1915,
  1847,
  3388],
 [3953, 1473, 3364, 4244, 2826, 3492, 1110, 3410, 1948, 4209, 3656],
 [1253, 3235, 2727, 2219, 1304, 1006, 1538, 1093, 1948, 4209, 3656],
 [3650, 2726, 4446, 4786, 2258, 866, 615, 2671, 1538, 3265],
 [4521, 618, 531, 896, 472, 1016, 417, 3210],
 [3134, 3733, 1186, 3144, 559, 1789, 3485, 4736, 3248, 3775, 2815],
 [2379, 4764, 2447, 866, 1538, 1304],
 [1628, 4997, 2237, 536, 4545, 4461, 98, 1948, 4209, 3656],
 [513, 4091, 4267, 2645, 4033, 1948, 4209, 3656],
 [2450, 4746, 2754, 3888, 1847, 2104, 4402, 1290, 4584, 2108],
 [4685, 2611, 3514],
 [1251, 698, 3563, 4487, 1538, 1220, 1144, 3388],
 [4230, 3596, 2495, 3638, 2356, 1422, 3722, 223, 3896],
 

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1013 2515 4403]
 [   0    0    0 ... 2514  908 3388]
 [   0    0    0 ... 4415 1959 1680]
 ...
 [   0    0    0 ... 3526  191 2916]
 [   0    0    0 ... 3682 3492 3388]
 [   0    0    0 ... 4434  954 3388]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4664,
       1754,  270, 3232, 2447,  714,  441, 1013, 2515, 4403], dtype=int32)

Creating model

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(3474, (3474,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((3474, 20), (3474,))

In [ ]:

 from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

 Finally Training

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
37/37 [==============================] - 2s 42ms/step - loss: 0.6135 - accuracy: 0.6863 - val_loss: 0.4887 - val_accuracy: 0.7847
Epoch 2/50
37/37 [==============================] - 1s 31ms/step - loss: 0.3211 - accuracy: 0.8590 - val_loss: 0.3670 - val_accuracy: 0.8082
Epoch 3/50
37/37 [==============================] - 1s 31ms/step - loss: 0.1494 - accuracy: 0.9446 - val_loss: 0.3405 - val_accuracy: 0.8413
Epoch 4/50
37/37 [==============================] - 1s 30ms/step - loss: 0.0916 - accuracy: 0.9729 - val_loss: 0.2920 - val_accuracy: 0.8684
Epoch 5/50
37/37 [==============================] - 1s 30ms/step - loss: 0.0465 - accuracy: 0.9888 - val_loss: 0.3540 - val_accuracy: 0.8500
Epoch 6/50
37/37 [==============================] - 1s 30ms/step - loss: 0.0235 - accuracy: 0.9948 - val_loss: 0.3904 - val_accuracy: 0.8614
Epoch 7/50
37/37 [==============================] - 1s 31ms/step - loss: 0.0144 - accuracy: 0.9974 - val_loss: 0.5394 - val_accuracy: 0.8396
Epoch 8/50
37

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout

Creating model

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[592,  35],
       [393, 127]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6268526591107236